

# 印加宝藏
<!-- ![印加宝藏](inca.png){: style="max-width: 100%; height: auto; display: block; margin: 0 auto;" } -->

印加宝藏是一个经典的桌游，主要考验玩家的策略规划和资源管理能力。在游戏中，玩家共同进入祭坛或矿道，逐张抽取宝物与陷阱卡，依次决定“继续冒险”或“撤离带宝”。若触发重复陷阱，留下的玩家将失去本轮所有宝物；而撤离者则安全存入帐篷，并且单独撤退者还可获取神器。

## 极致简化版
在极致简化版中，玩家只需抽取宝物卡和陷阱卡，所有的宝物卡得分都设为1分。此外，为了避免陷阱卡的复杂性，我们假设只存在一张陷阱卡，一旦抽取就会触发陷阱。最后，我们先假设只有一名玩家参与游戏。

### 问题描述

- 输入
  - 给定n张卡片
    - n-1张得分为1的宝物卡
    - 1张陷阱卡
  - 已经翻开的卡片数量i
- 输出
  - 玩家是否应该继续冒险

### 建模

我们定义在第$i$张卡片翻开后，玩家继续冒险的期望收益为$E(i,continue)$，撤离的期望收益为$E(i,leave)$。当$E(i,continue) > E(i,leave)$时，玩家应该继续冒险；否则，玩家应该撤离。

- 如果玩家选择撤离，期望收益为$E(i,leave) = i$（因为已经翻开的卡片都是宝物卡，得分为1）。
- 如果玩家继续冒险，下一张卡片可能是陷阱卡或宝物卡：
  - 如果是陷阱卡，玩家将失去所有宝物，期望收益为0。
  - 如果是宝物卡，需要考虑剩余卡片的情况：
    - 继续冒险的期望收益为$E(i+1, continue)$（当前宝物得分加上后续的期望收益）。
    - 撤离的期望收益为$E(i+1, leave)$（当前宝物得分加上后续的期望收益）。

接下来，我们分析在第$i$张卡片翻开时，发生陷阱卡的概率为$\frac{1}{n-i}$，发生宝物卡的概率为$\frac{n-1-i}{n-i}$。
因此，继续冒险的期望收益可以表示为：
$$
E(i, continue) = \frac{1}{n-i} \cdot 0 + \frac{n-1-i}{n-i} \cdot \max(E(i+1, continue), E(i+1, leave))
$$

公式太复杂了，让我们换个方式来表达吧，
我们进一步定义$E(i)=\max(E(i, continue), E(i, leave))$，即在第$i$张卡片翻开时的最大期望收益。
因此，我们可以将期望收益的递推关系简化为：

$$
E(i) = \max(i, \frac{n-1-i}{n-i} \cdot E(i+1))
$$

也就是说，对于当前的第$i$张卡片，我们需要考虑未来的所有可能性，并计算出继续冒险和撤离的期望收益。

**Observation**: 当我们翻到第$n-1$张卡片时，如果游戏还在继续，说明最后一张卡片必然是陷阱卡，因此$E(n-1, continue) = 0$。这是最后，也是我们递推的开始。

``来一首反方向的钟``

In [18]:
import numpy as np

n = 10

# con = [0] * n
con = np.zeros(n, dtype=float)
ex= np.zeros(n, dtype=float)
con[n-1] = 0
for i in range(n-2, -1, -1):
    # con[i] = (n-1-i)/(n-i)*max(con[i+1], i+1)
    con[i] = (n-1-i)/(n-i) * ex[i+1] 
    ex[i]=max(con[i],i)
print (con)

leave = np.arange(0, n, dtype=float)
print(leave>con)
# 输出第一个大于con的索引
index = np.argmax(leave > con)
print(index)

[2.5        2.77777778 3.125      3.57142857 4.16666667 4.8
 5.25       5.33333333 0.         0.        ]
[False False False False False  True  True  True  True  True]
5




## 扩展陷阱卡

在游戏中，第一次翻出陷阱卡时，玩家并不会失去所有宝物，而只是得到了提示。当翻出第二张陷阱卡时，玩家才会失去所有宝物。

### 问题描述
当玩家翻开第$i$张卡片时，陷阱卡已经出现了$j$次。我们需要判断玩家是否应该继续冒险。

### 建模
跟着上面的思路，我们可以定义：
- $E(i, j)$：在第$i$张卡片翻开时，陷阱卡已经出现$j$次的期望收益。

很容易分析到：
$$
\forall j \geq 2, E(i, j) = 0
$$
因为当陷阱卡出现两次时，玩家已经失去所有宝物。

对于$j = 0$和$j = 1$的情况，我们可以使用类似的递推关系：

$$
E(i, j, continue) = \frac{3-j}{n-i} \cdot E(i+1, j+1) + \frac{n-i-3+j}{n-i} \cdot E(i+1, j)
$$

$$

E(i, j) = \max\left(
    E(i, j, continue), 
    i
\right)

$$

In [ ]:
import numpy as np
n=10
con = np.zeros([n,3], dtype=float)
ex = np.zeros([n,3], dtype=float)
leave = np.arange(0, n, dtype=float)
leave = np.tile(leave, (3, 1)).T
# print(leave)

for i in range(n-1, -1, -1):
    for j in range(3-2, -1, -1):
        con[i][j] = (3-j)/(n-i)*ex[i+1][j+1] + (n-i-3+j)/(n-i)*ex[i+1][j] 
        ex[i][j] = max(con[i][j], leave[i][j])
print(con)
print(ex)
print(leave < con)


IndexError: index 10 is out of bounds for axis 0 with size 10